## Step 1: Import necessary modules

In [1]:
!pip install -q -U tensorflow numpy


[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
import numpy as np

from tensorflow.keras.datasets import mnist

2023-05-16 11:06:23.673043: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Step 2: Load and prepare the MNIST dataset

MNIST data is a collection of hand-written digits that contains 60,000 examples for training and 10,000 examples for testing. The digits have been size-normalized and centered in a fixed-size image (28x28 pixels) with values from 0 to 255. 

In [22]:
# Load data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Convert to float32
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)

# Flatten images to vectors of length 784 (28*28)
x_train, x_test = x_train.reshape([-1, 784]), x_test.reshape([-1, 784])

# Normalize the pixel intensities to make sure their values are between 0 to 1
# by dividing them by 255
x_train, x_test = x_train / 255., x_test / 255.

## Step 3: Set up hyperparameters etc.

- num_classes denotes the number of outputs, which is 10, as we have digits from 0 to 9 in the data set. 
- num_features defines the number of input parameters, and we store 784 since each image contains 784 pixels.

- learning_rate defines the step size the model should take to converge to a minimum loss
- training_steps defines the number of steps the model will take to train itself completely
- batch_size denotes the number of samples per each batch in the training process
- display_step to iterate over the training steps and print them in the training process

In [4]:
# MNIST dataset parameters
num_classes = 10
num_features = 784

# Training parameters
learning_rate = 0.01
training_steps = 1000
batch_size = 256
display_step = 50

## Step 4: Shuffle and batch data

Shuffle and batch the data before we start the actual training to avoid the model from getting biased by the data. This will allow our data to be more random and helps our model to gain higher accuracies with the test data.

In [5]:
# Get the slices of an array in the form of objects
train_data=tf.data.Dataset.from_tensor_slices((x_train,y_train))

# Shuffle and batch the data
train_data=train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

## Step 5: Initialise weights and biases

In [6]:
# Weight of shape [784, 10], the 28*28 image features, and a total number of classes.
W = tf.Variable(tf.ones([num_features, num_classes]), name="weight")

# Bias of shape [10], the total number of classes.
b = tf.Variable(tf.zeros([num_classes]), name="bias")

## Step 6: Define logistic regression and cost function

1. Define a logistic regression function which converts the inputs into a probability distribution proportional to the exponents of the inputs using the softmax function
2. Encode the outputs using the function tf.one_hot. We also define and compute the cross-entropy function as the loss function, which is given as cross-entropy loss = -ytrue*(log(ypred)) using tf.reduce_mean and tf.reduce_sum

In [7]:
def logistic_regression(x):
    return tf.nn.softmax(tf.matmul(x, W) + b)

def cross_entropy(y_pred, y_true):
    y_true = tf.one_hot(y_true, depth=num_classes)
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

## Step 7: Define accuracy calculator and optimizer

In [8]:
def accuracy(y_pred, y_true):
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

optimizer = tf.optimizers.SGD(learning_rate)

## Step 8: Optimization process

1. Define a run_optimization() function where we update the weights of our model. 
    - Calculate the predictions using the logistic_regression(x) method by taking the inputs and find out the loss generated by comparing the predicted value and the original value present in the data set. 
    - Compute the gradients using and update the weights of the model with our stochastic gradient descent optimizer.

In [9]:
def run_optimization(x, y):
# Wrap computation inside a GradientTape for automatic differentiation.
    with tf.GradientTape() as g:
        pred = logistic_regression(x)
        loss = cross_entropy(pred, y)
    # Compute gradients.
    gradients = g.gradient(loss, [W, b])
    # Update W and b following gradients.
    optimizer.apply_gradients(zip(gradients, [W, b]))

## Step 9: Training loop

In [32]:
enumerate(train_data.take(5, 0))

2023-05-16 11:21:51.335131: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [60000]
	 [[{{node Placeholder/_1}}]]
2023-05-16 11:21:51.335599: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [60000,784]
	 [[{{node Placeholder/_0}}]]


In [30]:
train_data

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 784), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.uint8, name=None))>

In [28]:
for image, labels in train_data.take(5):
    print(labels)

tf.Tensor(
[4 6 4 3 4 6 2 7 2 8 5 0 0 7 1 3 8 6 8 9 7 1 5 0 3 8 5 2 2 0 8 0 0 2 1 5 1
 0 3 7 5 7 3 3 5 6 6 3 7 4 9 6 9 8 7 9 8 8 7 0 6 6 7 4 2 0 3 1 2 1 4 2 0 1
 4 0 4 1 8 1 7 9 6 3 5 8 6 9 6 4 6 6 8 3 8 2 3 5 6 8 5 4 7 0 0 3 0 2 2 9 9
 9 3 0 2 7 1 7 2 7 4 2 5 4 1 0 0 5 5 1 0 7 8 1 8 9 1 6 1 3 8 3 1 4 2 3 7 1
 1 9 2 7 2 7 0 2 0 8 6 3 6 4 4 4 8 2 2 6 6 7 9 9 9 3 7 1 6 1 2 1 6 3 4 1 1
 8 5 6 9 1 1 0 7 6 4 3 1 7 4 1 2 1 9 0 8 0 4 3 6 1 7 2 3 5 3 2 6 9 2 5 0 1
 0 7 3 8 8 5 1 6 7 4 9 6 9 4 1 9 7 4 2 8 3 2 6 8 1 4 3 2 5 8 2 4 7 2], shape=(256,), dtype=uint8)
tf.Tensor(
[2 3 3 9 8 6 2 7 6 7 0 1 6 4 0 2 5 2 7 3 4 3 6 1 9 8 4 5 6 6 2 9 3 5 9 8 9
 8 7 7 7 3 6 5 3 6 7 9 2 2 7 5 5 7 2 0 3 5 3 0 7 9 1 1 9 3 1 8 1 8 9 7 3 1
 1 6 4 2 1 8 6 2 1 9 0 5 5 9 4 4 1 4 4 3 4 3 7 8 9 2 3 6 1 7 9 6 4 4 4 7 2
 7 3 9 4 1 2 6 5 4 4 4 2 8 9 3 0 4 4 0 4 3 8 1 4 9 4 6 6 1 9 1 1 8 7 6 1 0
 6 2 7 5 4 0 6 3 2 9 4 1 7 6 2 3 9 8 4 0 9 4 2 1 1 6 8 5 8 2 8 3 9 0 7 5 5
 3 8 6 0 1 4 8 6 0 7 9 2 1 9 0 2 3 1 3 4 2 7 8 5 4 0 3 

2023-05-16 11:20:10.367792: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [60000]
	 [[{{node Placeholder/_1}}]]
2023-05-16 11:20:10.368220: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [60000,784]
	 [[{{node Placeholder/_0}}]]


In [33]:
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps, 0)):
    run_optimization(batch_x, batch_y)
    if step % display_step == 0:
        pred = logistic_regression(batch_x)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

2023-05-16 11:22:01.941207: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [60000,784]
	 [[{{node Placeholder/_0}}]]
2023-05-16 11:22:01.941699: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [60000]
	 [[{{node Placeholder/_1}}]]


step: 0, loss: 614.903625, accuracy: 0.460938
step: 50, loss: 595.460510, accuracy: 0.773438
step: 100, loss: 624.926392, accuracy: 0.785156
step: 150, loss: 70.205956, accuracy: 0.941406
step: 200, loss: 207.158386, accuracy: 0.824219
step: 250, loss: 67.996414, accuracy: 0.929688
step: 300, loss: 90.952713, accuracy: 0.917969
step: 350, loss: 158.530014, accuracy: 0.843750
step: 400, loss: 163.232483, accuracy: 0.855469
step: 450, loss: 25.215725, accuracy: 0.960938
step: 500, loss: 114.201691, accuracy: 0.878906
step: 550, loss: 53.076317, accuracy: 0.941406
step: 600, loss: 92.225449, accuracy: 0.898438
step: 650, loss: 75.926262, accuracy: 0.914062
step: 700, loss: 61.724026, accuracy: 0.925781
step: 750, loss: 202.819153, accuracy: 0.847656
step: 800, loss: 85.210052, accuracy: 0.917969
step: 850, loss: 45.227894, accuracy: 0.933594
step: 900, loss: 154.704849, accuracy: 0.835938
step: 950, loss: 98.358047, accuracy: 0.902344


## Test accuracy

In [34]:
pred = logistic_regression(x_test)

print("Test Accuracy: %f" % accuracy(pred, y_test))

Test Accuracy: 0.839900
